# Block Based Motion Estimation

In [ ]:
import numpy as np
import frame
import YCoCg as YUV
import motion
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import LP
import cv2 as cv

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_frame(frame, prefix=None):
    #frame = normalize(frame)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(frame)

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circles/", 0))[...,0]
P = YUV.from_RGB(frame.read("../sequences/moving_circles/", 1))[...,0]

In [ ]:
flow = motion.estimate(P, R)

In [ ]:
flow.shape

In [ ]:
flow.min()

In [ ]:
for i in range(64):
    for j in range(64):
        print((np.rint(flow)[i,j][1]), end=' ')
    print()

In [ ]:
show_frame(flow[...,0])

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)
P = frame.read("../sequences/moving_circles/", 1)
prediction = motion.make_prediction(R, flow)
show_frame(prediction)
show_frame(P)

In [ ]:
show_frame(P-prediction)

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circles/", 0))[...,0]
P = YUV.from_RGB(frame.read("../sequences/moving_circles/", 1))[...,0]

In [ ]:
#      5       3       8
#      1       0       2
#      7       4       6
# (-1,-1) (-1, 0) (-1, 1)
# ( 0,-1) ( 0, 0) ( 0, 1)
# ( 1,-1) ( 1, 0) ( 1, 1)
#spiral  = np.array([(-1,-1), (-1, 1), ( 1,-1), ( 1, 1), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1), ( 0, 0)])
spiral  = np.array([( 0, 0), (-1,-1), (-1, 1), ( 1,-1), ( 1, 1), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1)])
spiral  = np.array([( 0, 0), ( 0,-1), ( 0, 1), (-1, 0), ( 1, 0), (-1,-1), ( 1, 1), ( 1,-1), (-1, 1)])

# (0,0) (0,1) (0,2)
# (1,0) (1,1) (1,2)
# (2,0) (2,1) (2,2)
spiral_ = np.array([( 1, 1), ( 0, 0), ( 0, 2), ( 2, 0), ( 2, 2), ( 0, 1), ( 2, 1), ( 1, 0), ( 1, 2)])
spiral_ = np.array([( 1, 1), ( 1, 0), ( 1, 2), ( 0, 1), ( 2, 1), ( 0, 0), ( 2, 2), ( 2, 0), ( 0, 2)])

In [ ]:
# (x, y)
#      5       3       8
#      1       0       2
#      7       4       6
# (-1,-1) ( 0,-1) ( 1,-1)
# (-1, 0) ( 0, 0) ( 1, 0)
# (-1, 1) ( 0, 1) ( 1, 1)
spiral  = np.array([( 0, 0), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1), (-1,-1), ( 1, 1), (-1, 1), ( 1,-1)], dtype=np.int16)
#                        0        1        2        3        4        5        6        7        8

# (0,0) (1,0) (2,0)
# (0,1) (1,1) (2,1)
# (0,2) (1,2) (2,2)
spiral_ = np.array([( 1, 1), ( 0, 1), ( 2, 1), ( 1, 0), ( 1, 2), ( 0, 0), ( 2, 2), ( 0, 2), ( 2, 0)], dtype=np.int16)

In [ ]:
print(spiral.dtype)

In [ ]:
extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = R

In [ ]:
#extended_R = np.pad(R, (1, 1), 'edge') # Ojo, no funciona

In [ ]:
show_frame(extended_R)

In [ ]:
D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)

In [ ]:
for i in range(9):
    diff = extended_R[spiral_[i][1]:R.shape[0]+spiral_[i][1], spiral_[i][0]:R.shape[1]+spiral_[i][0]] - P
    D[:,:,i] = diff*diff

In [ ]:
print(D[0,0,0], D[0,0,1], D[0,0,2], D[0,0,3])

In [ ]:
DD = np.empty_like(D).astype(np.float32)
DD[...] = D
print(D[0,0,1])
show_frame(D[...,0])
kernel = np.ones((15,15),np.float32)/(15*15)
D = cv.filter2D(DD,-1,kernel)
#show_frame(D[...,0]+D[...,1])
show_frame(np.rint(D[...,0]))


In [ ]:
flow_indexes = np.argmin(np.rint(D), axis=2)
for i in range(64):
    for j in range(64):
        print(flow_indexes[i,j], end='')
    print()

In [ ]:
show_frame(flow_indexes)

In [ ]:
print(D[0,0,0], D[0,0,1], D[0,0,2], D[0,0,3])

In [ ]:
flow = spiral[flow_indexes]

In [ ]:
show_frame(D[:,:,1] - D[:,:,8])

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)

In [ ]:
show_frame(R)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction)

In [ ]:
P = frame.read("../sequences/moving_circles/", 1)
show_frame(P)

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circles/", 0))
P = YUV.from_RGB(frame.read("../sequences/moving_circles/", 1))

In [ ]:
def spiral_estimate(P, R, window_side=15):
    print("window_side =", window_side)
    extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
    extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = R
    print("extended_R.shape", extended_R.shape)
    #extended_R = np.pad(R, (1, 1), 'edge')
    #print("extended_R.shape", extended_R.shape)
    D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)
    for i in range(9):
        diff = extended_R[spiral_[i][1] : R.shape[0] + spiral_[i][1],
                          spiral_[i][0] : R.shape[1] + spiral_[i][0]] - P
        D[:,:,i] = diff*diff
    # Tenemos la distancia L2, pixel-wise, para cada posible desplazamiento de ambas matrices, R y P.
    # Ahora se trata de encontrar una nueva matriz de distancias donde se tenga en cuenta las distancias adyacentes.
    DD = np.empty_like(D).astype(np.float32)
    DD[...] = D
    kernel = np.ones((window_side,window_side),np.float32)/(window_side*window_side)
    D = cv.filter2D(DD, -1, kernel)
    #show_frame(D[...,0]+D[...,1])
    #show_frame(D[...,0])
    #for y in range(D.shape[0]):
    #    for x in range(D.shape[1]):
    #        print(D[y,x], end=' ')
    #    print()
    #print(80*"=")
    flow_indexes = np.argmin(np.rint(D), axis=2)
    for i in range(flow_indexes.shape[0]):
        for j in range(flow_indexes.shape[1]):
            print(flow_indexes[i,j], end='')
        print()
    print(80*"=")
    flow = spiral[flow_indexes]
    return flow

In [ ]:
flow = spiral_estimate(P[...,0], R[...,0])

In [ ]:
def spiral_estimate2(initial_flow, P, R, window_side=15):
    print("window_side =", window_side)
    print("inital_flow.dtype", initial_flow.dtype)
    print("R.shape", R.shape)
    print("initial_flow.shape", initial_flow.shape)
    extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
    extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = motion.make_prediction(R, initial_flow)
    D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)
    for i in range(9):
        diff = extended_R[spiral_[i][1] : R.shape[0] + spiral_[i][1],
                          spiral_[i][0] : R.shape[1] + spiral_[i][0]] - P
        D[:,:,i] = diff*diff
    # Tenemos la distancia L2, pixel-wise, para cada posible desplazamiento de ambas matrices, R y P.
    # Ahora se trata de encontrar una nueva matriz de distancias donde se tenga en cuenta las distancias adyacentes.
    DD = np.empty_like(D).astype(np.float32)
    DD[...] = D
    kernel = np.ones((window_side,window_side),np.float32)/(window_side*window_side)
    D = cv.filter2D(DD, -1, kernel)
    #show_frame(D[...,0]+D[...,1])
    show_frame(D[...,0])
    #for y in range(D.shape[0]):
    #    for x in range(D.shape[1]):
    #        print(D[y,x], end=' ')
    #    print()
    #print(80*"=")
    flow_indexes = np.argmin(np.rint(D), axis=2)
    for i in range(flow_indexes.shape[0]):
        for j in range(flow_indexes.shape[1]):
            print(flow_indexes[i,j], end='')
        print()
    print(80*"=")
    print(spiral[flow_indexes].shape)
    print(initial_flow.shape)
    flow = initial_flow + spiral[flow_indexes]
    return flow

In [ ]:
flow = np.zeros((R.shape[0], R.shape[1], 2), dtype=np.int16)
print("flow.shape", flow.shape)
flow = spiral_estimate2(flow, P[...,0], R[:,:,0])

In [ ]:
# Funciona para log2_search_area = 0 y 1 (search_area 1 y 2)
def estimate(P, R, log2_search_area, n_levels):
    if log2_search_area > 0:
        P_pyramid = LP.analyze(P, log2_search_area)
        R_pyramid = LP.analyze(R, log2_search_area)
    else:
        P_pyramid = [P]
        R_pyramid = [R]
    #total_flow = np.zeros(((R_pyramid[0].shape[0]+1)//2, (R_pyramid[0].shape[1]+1)//2, 2), dtype=np.int16)
    total_flow = np.zeros((R_pyramid[0].shape[0], R_pyramid[0].shape[1], 2), dtype=np.int16)
    for l in range(n_levels):
        _R_pyramid = motion.make_prediction(R_pyramid[l], total_flow.astype(np.float32))
        flow = spiral_estimate(P_pyramid[l], _R_pyramid)
        total_flow *= 2
        total_flow += flow
        total_flow = total_flow.astype(np.int16)
        #for y in range(total_flow.shape[0]):
        #    for x in range(total_flow.shape[1]):
        #        print(total_flow[y,x], end=' ')
        #    print()
        #print(80*"=")
        #show_frame(R_pyramid[l+1])
        R_pyramid[l+1] = LP.synthesize_step(R_pyramid[l], R_pyramid[l+1])
        P_pyramid[l+1] = LP.synthesize_step(P_pyramid[l], P_pyramid[l+1])
        total_flow = LP.synthesize_step(total_flow.astype(np.float32), np.zeros(total_flow.shape)).astype(np.int16)
    _R = motion.make_prediction(R, total_flow.astype(np.float32))
    flow = spiral_estimate(P, _R)
    if log2_search_area > 0:
        #total_flow = LP.synthesize_step(total_flow.astype(np.float32), np.zeros(total_flow.shape)).astype(np.int16)
        #total_flow *= 2
        total_flow += flow
        total_flow = total_flow.astype(np.int16)
    else:
        return flow
    return total_flow

In [ ]:
flow = estimate(P[...,0], R[...,0], 0, 0)

In [ ]:
def local_estimate(P, R):
    flow = np.zeros((R.shape[0], R.shape[1], 2), dtype=np.int16)
    flow = spiral_estimate(P, R)
    return flow

In [ ]:
local_estimate(P[...,0], R[...,0])

In [ ]:
def estimate2(P, R, log2_search_area, n_levels):
    w_side = 15
    if log2_search_area > 0:
        P_pyramid = LP.analyze(P, log2_search_area)
        R_pyramid = LP.analyze(R, log2_search_area)
    else:
        P_pyramid = [P]
        R_pyramid = [R]
    #total_flow = np.zeros(((R_pyramid[0].shape[0]+1)//2, (R_pyramid[0].shape[1]+1)//2, 2), dtype=np.int16)
    flow = np.zeros((R_pyramid[0].shape[0], R_pyramid[0].shape[1], 2), dtype=np.int16)
    for l in range(n_levels):
        #_R_pyramid = motion.make_prediction(R_pyramid[l], total_flow.astype(np.float32))
        #show_frame(_R_pyramid, f"R_pyramid[{l}]")
        show_frame(R_pyramid[l], f"R_pyramid[{l}]")
        show_frame(P_pyramid[l], f"P_pyramid[{l}]")
        flow = spiral_estimate2(flow, P_pyramid[l], R_pyramid[l], window_side = w_side >> (log2_search_area - l))
        #total_flow *= 2
        #total_flow += flow
        #total_flow = total_flow.astype(np.int16)
        #for y in range(total_flow.shape[0]):
        #    for x in range(total_flow.shape[1]):
        #        print(total_flow[y,x], end=' ')
        #    print()
        #print(80*"=")
        #show_frame(R_pyramid[l+1])
        R_pyramid[l+1] = LP.synthesize_step(R_pyramid[l], R_pyramid[l+1])
        P_pyramid[l+1] = LP.synthesize_step(P_pyramid[l], P_pyramid[l+1])
        flow = LP.synthesize_step(flow.astype(np.float32), np.zeros(flow.shape)).astype(np.int16)
    #_R = motion.make_prediction(R, total_flow.astype(np.float32))
    show_frame(R, f"_R")
    show_frame(P, f"P")
    flow = spiral_estimate2(flow, P, R, window_side = w_side)
    #total_flow = total_flow.astype(np.int16)
    return flow

In [ ]:
flow = estimate2(P[...,0], R[...,0], 2, 2)

In [ ]:
show_frame(flow[...,0])

In [ ]:
show_frame(flow[...,1])

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)
P = frame.read("../sequences/moving_circles/", 1)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction)

In [ ]:
show_frame(P)

In [ ]:
show_frame(R)

In [ ]:
show_frame(prediction-P)

## Ahora probamos con un desplazamiento de 2

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0))
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 2))

In [ ]:
flow = estimate2(P[...,0], R[...,0], 1, 1)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(prediction-P)

## Ahora probamos con un desplazamiento de 3

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0))
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 3))

In [ ]:
flow = estimate2(P[...,0], R[...,0], 2, 2)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(prediction-P)

## Ahora probamos con un desplazamiento de 4

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0))
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 4))

In [ ]:
flow = estimate(P[...,0], R[...,0], 2, 2)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(prediction-P)

In [ ]:
############################3

In [ ]:
D = []
for i in _spiral:
    difference = extended_R[i[0]:R.shape[0]+i[0], i[1]:R.shape[1]+i[1]] - P
    D.append(difference)

In [ ]:
D[1].max()